In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv(r'baseball.csv')

In [2]:
df.columns

Index(['Title'], dtype='object')

In [3]:
df1=pd.DataFrame()
print(df1.columns)
df1['Title']=df['Title'].apply(lambda x:x.lower())
df1['Year']=df1['Title'].str.extract( "(\d{4})",expand=False ).str.strip()
# df1['Year].isnull().values.any()   #True
# df1['Year'].isnull().values.sum()  #7163
# perc_year_na=7163/len(df)          #0.3%

Index([], dtype='object')


In [4]:
df1["Unmatched"] = df1['Title'].apply(lambda x: x.lower())

In [5]:
TeamName=['Braves','Marlins','Mets','Phillies','Nationals','Cubs','Reds','Brewers','Pirates','Cardinals','Diamondbacks',
'Rockies','Dodgers','Padres','Giants','Orioles','Red Sox','Yankees','Rays','Blue Jays','White Sox','Indians',
'Tigers','Royals','Twins','Astros','Angels','Athletics','Mariners','Ranger','yankees','Red Stocking','White Stocking','Brown Stocking','Greys','Browns',
'Bees','Redlegs','Beaneaters','Brewers','Senators','Robins','Doves','Rustlers','Orphans','Naps','Hightlanders','Colts','Americans','Bridgerooms','Superbas',
'Perfectos',"Chiefs","Sting","Sting","Mustangs","Generals","Black Sox","Admirals","Renegades","Capitals","Selects","Predators","Heat","Smoke","Aces",
"Reapers","Gremlins","Gators","Axemen","Falcons","Monsters","Legends","Dukes","Hammer","Outlaws","Redmen","Stix","Force","Merchants","Bandits","Hawks","Knights"
,"Bullets","Deuces","Jacks","Diamond Jaxx","K-Men","Magicians","Sluggers","Chuckers","Fireballers","Phantoms","Matadors","Sharks"
,"Bulls","Hounds","Wildcats","Kraken","Kolts","Blaze","Kings","Classics","Klassics"]
TeamName = [i.lower() for i in TeamName]

In [6]:
def team_name_extract(x):
    for i in TeamName:
        if i in x.lower():
            return i

In [7]:
df1['TeamName']=df1['Title'].apply(lambda x:team_name_extract(x))
df1['TeamName'].isnull().values.sum()/len(df) #missing 95% team name is not a frequently used key words in basketball cards

0.7632527013340228

In [8]:
pd.DataFrame(df1.groupby('TeamName')['TeamName'].count()).rename(columns={"TeamName": "Count"}).sort_values(by='Count',ascending=False)

,Count
TeamName,
yankees,59546
braves,45615
dodgers,32538
angels,29931
mets,21501
...,...
merchants,5
hounds,4
diamond jaxx,3


# Professional Grader

In [9]:
df1['Grader']=df1['Title'].str.extract(r'(psa|bgs|sgc|bvg|bccg|scd|gai|sgv)',expand=True)
#df1['Grader']=df1['Grader'].str.upper()

In [10]:
pd.DataFrame(df1.groupby('Grader')['Grader'].count()).rename(columns={"Grader": "Count"}).sort_values(by='Count',ascending=False)

,Count
Grader,
psa,374586
bgs,89750
sgc,14679
bvg,3815
bccg,1814
gai,894
scd,78


# Classifier

In [11]:
df1['Classifier']=df1['Title'].str.extract(r'(single|lot|set|box|pack|team set|uncut sheet|box&case breaks|cello|booklet)',expand=True)
#df1['Classifier']=df1['Classifier'].str.capitalize()

In [12]:
# df1['Classifier']=df['Title'].str.extract(r'(Single|Lot|Set|Box|Pack|Team Set|Uncut Sheet|Box&Case Breaks|Cello|single|lot|set|box|pack|team set|uncut sheet|box&case breaks|cello|SINGLE|LOT|SET|BOX|PACK|TEAM SET|UNCUT SHEET|BOX&CASE BREAKS|CELLO)',expand=True)
# df1['Classifier']=df1['Classifier'].str.capitalize()

In [13]:
pd.DataFrame(df1.groupby('Classifier')['Classifier'].count()).rename(columns={"Classifier": "Count"}).sort_values(by='Count',ascending=False)

,Count
Classifier,
lot,140995
box,69187
set,52238
pack,14242
team set,5975
single,4025
booklet,2798
cello,890
uncut sheet,85


# Card Number

In [14]:
df1['CardNumber']=df1['Title'].str.extract("#(\w*\.?\-?\/?\d?\w*\-?\w+)",expand=True )
#df1['CardNumber']=df['Title'].str.extract( "#(\w*\.?\-?\d?\w+)",expand=True )
 #df1['CardNumber'].isnull().values.sum()/len(df) # 42% missing

# Quantity Made

In [15]:
df1['QuantityMade']=df1['Title'].str.extract( "(\d?\d?\d?\d?\d?\d?\/\d+)",expand=True )
#df1['QuantityMade'].isnull().values.sum()/len(df) # 74% missing

# Specialty

In [16]:
Keywords= ["jerseys","jersey","freshman","rookies","rc", "mvp","mvps","hof","auto","rookie","nnof","xrc","big four","bip","foil","jsy","nib","pc","opg"]

def Special(x):
    String=""
    for i in Keywords:
        for k in x.split(" "):
            if i in k:
                String += str(i) + ","
    return String[:-1]

df1['Specialty']=df1['Title'].apply(lambda x: Special(x))

In [17]:
df1['Specialty'].isnull().values.sum()/len(df) #%50.94 missing   

0.0

# Players

In [18]:
import requests
from bs4 import BeautifulSoup
def fetch_famous_players(baseurl):
    famous_player = []
    response = requests.get(baseurl)
    if response.status_code == 200:
        print("Success")
    else:
        print("Failure")
    results_page = BeautifulSoup(response.content,'html.parser')
    page = results_page.find("table").find_all('td', attrs={'style':'font-family: Arial, sans-serif; color: windowtext; font-size: 10.0pt; font-weight: 400; font-style: normal; text-decoration: none; text-align: general; vertical-align: bottom; white-space: nowrap; border: medium none; padding-left: 1px; padding-right: 1px; padding-top: 1px'})
    for i in page:
        famous_player.append(i.text.split('\n')[1].strip())
    return famous_player

In [19]:
famous_player = fetch_famous_players("http://ainsworthsports.com/baseball_player_rankings_all_time_1_to_1000.htm")
famous_player.extend(fetch_famous_players("http://ainsworthsports.com/baseball_player_rankings_all_time_1001_to_2000.htm"))
famous_player.extend(fetch_famous_players("http://ainsworthsports.com/baseball_player_rankings_all_time_2001_to_3000.htm"))
famous_player.extend(fetch_famous_players("http://ainsworthsports.com/baseball_player_rankings_all_time_3001_to_4000.htm"))
famous_player.extend(fetch_famous_players("http://ainsworthsports.com/baseball_player_rankings_all_time_4001_to_5000.htm"))
type(famous_player)


Success
Success
Success
Success
Success


list

In [99]:
def person(x):
    for i in famous_player:
        dummy = i.lower()
        if dummy in x:
            return i

In [100]:
df1['Player_Name']=df1['Title'].apply(lambda x: person(x))

In [101]:
pd.DataFrame(df1.groupby('Player_Name')['Player_Name'].count()).rename(columns={"Player_Name": "Count"}).sort_values(by='Count',ascending=False)

,Count
Player_Name,
Ronald Acuna,67308
Mike Trout,55008
Derek Jeter,37374
Fernando Tatis,33385
Vladimir Guerrero,29112
...,...
Jap Barbeau,1
Ray Sanders,1
Brad Arnsberg,1


In [102]:
df1

,Title,Year,Unmatched,TeamName,Grader,Classifier,CardNumber,QuantityMade,Specialty,Player_Name,Manufacturer,Rating,num_words_unmatched,num_words_title,matched_percentage
0,1974 laughlin all-star games phil cavarretta #44,1974,all-star games,None,NaN,NaN,44,NaN,,Phil Cavarretta,Laughlin,NaN,2,7,0.714286
1,1955 bowman del crandall #217,1955,,None,NaN,NaN,217,NaN,,Del Crandall,Bowman,NaN,0,5,1.000000
2,1957 topps del crandall #133,1957,,None,NaN,NaN,133,NaN,,Del Crandall,Topps,NaN,0,5,1.000000
3,1995 skybox emotion timeless cal ripken jr #8 hof,1995,emotion timeless jr,None,NaN,box,8,NaN,hof,Cal Ripken,SkyBox,NaN,3,9,0.666667
4,1989 st vincent rookies stamps ken griffey jr ...,1989,st vincent stamps jr,None,NaN,NaN,NaN,NaN,"rookies,hof,rookie,rookie",None,NaN,NaN,4,10,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293584,shohei ohtani 2019 bowman sterling black atomi...,2019,sterling black atomic refractor graph,None,NaN,NaN,/10,/10,"auto,auto",None,Bowman SP,NaN,5,12,0.583333
2293585,shohei ohtani 2019 bowman sterling black atomi...,2019,sterling black atomic refractor graph,None,NaN,NaN,/10,/10,"auto,auto",None,Bowman SP,NaN,5,12,0.583333
2293586,shohei ohtani 2019 bowman sterling black atomi...,2019,sterling black atomic refractor graph,None,NaN,NaN,/10,/10,"auto,auto",None,Bowman SP,NaN,5,12,0.583333
2293587,shohei ohtani 2019 bowman sterling black atomi...,2019,sterling black atomic refractor graph,None,NaN,NaN,/10,/10,"auto,auto",None,Bowman SP,NaN,5,12,0.583333


In [103]:
df1['Player_Name'].isnull().sum()

762583

# Manufcturer

In [104]:
manufacturer=pd.read_csv(r'Sports_Cards_Keywords_Manufacturer.csv')
# manufacturer dictinoary
manufacturer = manufacturer.set_index('How It Appears').to_dict()['Manufacturuer']
print(len(manufacturer))
manufacturer_dict = {}
for key,val in manufacturer.items():
    if "|" in key:
        keys = key.split('|')
        for i in keys:
            manufacturer_dict[i.lower()] = val
    else:
        manufacturer_dict[key.lower()] = val
print(len(manufacturer))

442
442


In [105]:
manufacturer_list = list(manufacturer_dict.keys())
manufacturer_list = set(manufacturer_list)

In [106]:
def manufacturer_detect(x):
    res = ""
    for i in x.split(" "):
        if i in manufacturer_list:
            res = res + i + ","
    if res:
        return res
    else:
        return np.NaN

In [107]:
df1['Manufacturer']=df1['Unmatched'].apply(lambda x: manufacturer_detect(x))

In [108]:
df1['Manufacturer'].isnull().sum()

2293126

# Rating

In [109]:
rating=pd.read_csv(r'Sports_Cards_Keywords_Grade.csv')
# manufacturer dictinoary
rating = rating.set_index('How It May Appear').to_dict()['Grade']
print(len(rating))
rating_dict = {}
for key,val in rating.items():
    if "|" in key:
        keys = key.split('|')
        for i in keys:
            rating_dict[i.lower()] = val
    else:
        rating_dict[key.lower()] = val
print(len(rating))

75
75


In [110]:
print(len(rating_dict))
rating_list = list(rating_dict.keys())
rating_list.sort(reverse=True)

1581


In [111]:
def rating_detect(x):
    for i in rating_list:
        if i in x:
            return i

In [112]:
df1['Rating']=df1['Title'].apply(lambda x: rating_detect(x))

In [113]:
df1['Rating'].isnull().sum()

1476518

# Matching

In [114]:
def manufacturer_unmatched(x):
    if isinstance(x['Manufacturer'],str):
        dummy = x['Unmatched']
        manufacturer = x['Manufacturer'].split(',')
        while("" in manufacturer): 
            manufacturer.remove("")
        for i in manufacturer:
            dummy = dummy.replace(i,"")
        return dummy 
    else:
        return x['Unmatched']

In [115]:
df1['Unmatched']=df1.apply(lambda x: manufacturer_unmatched(x),axis = 1)

In [116]:
def rating_unmatched(x):
    if isinstance(x['Rating'],str):
        dummy = x['Unmatched']
        dummy = dummy.replace(x['Rating'],"")
        return dummy
    else:
        return x['Unmatched']

In [117]:
df1['Unmatched']=df1.apply(lambda x: rating_unmatched(x),axis = 1)

In [118]:
def else_unmatched(x):
    y = x['Unmatched']
    if isinstance(x['Player_Name'],str):
        name = x['Player_Name'].lower()
        y = y.replace(name,"")
    if isinstance(x['Year'],str):
        year = x['Year'].lower()
        y = y.replace(year,"")
    if isinstance(x['Specialty'],str):
        specialty = x['Specialty'].lower().split(',')
        for i in specialty:
            y = y.replace(i,"")
    if isinstance(x['Classifier'],str):
        classifiers = x['Classifier'].lower()
        y = y.replace(classifiers,"")
    if isinstance(x['QuantityMade'],str):
        quantity_made = x['QuantityMade'].lower()
        y = y.replace(quantity_made,"")
        if "/" in y:
            y = y.replace("/","")
    if isinstance(x['Grader'],str):
        garder = x['Grader'].lower()
        y = y.replace(garder,"")
    if isinstance(x['CardNumber'],str):
        card_number = x['CardNumber'].lower()
        y = y.replace(card_number,"")
        if "#" in y:
            y = y.replace("#","")
    if isinstance(x['TeamName'],str):
        team_name = x['TeamName'].lower()
        y = y.replace(team_name,"")
    return y

In [119]:
df1['Unmatched']=df1.apply(lambda x: else_unmatched(x),axis = 1)

In [120]:
def unmatched_trim(x):
    strings = x['Unmatched'].strip().split(' ')
    res = ' '.join([x for x in strings if x])
    return res

In [121]:
df1['Unmatched']=df1.apply(lambda x: unmatched_trim(x),axis = 1)

In [122]:
# !pip install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from collections import OrderedDict
nltk.download('punkt')
words_list = df1['Unmatched'].values.tolist()
striptext = " ".join(words_list)
words = word_tokenize(striptext)
lowercase_words = [word.lower() for word in words]
word_frequencies = FreqDist(lowercase_words)
most_frequent_words = FreqDist(lowercase_words).most_common(10000)
print(most_frequent_words)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chensiyuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('chrome', 430081), ('refractor', 276832), ('baseball', 211258), ('card', 183373), ('mint', 182501), ('jr', 162602), ('gem', 159617), ('graph', 141561), ('$', 135744), ('(', 134414), (')', 134210), ('*', 123561), ('update', 119909), ('of', 119193), ('draft', 109638), ('gold', 97585), ('prospects', 96318), ('blue', 82960), ("'s", 80218), ('heritage', 73561), ('ex', 66786), ('deck', 61189), ('upper', 61153), ('-', 56927), ('red', 54230), ('game', 52589), ('series', 50861), ('prospect', 50326), ('&', 50072), ('patch', 46880), ('edition', 46537), ('10', 46377), ('the', 42681), ('ex/', 41184), ('purple', 40884), ('green', 40162), ('stars', 39631), ('9.5', 39388), ('prizm', 39338), ('silver', 38800), ('black', 38395), ('high', 37922), ('2', 37782), ('ex/ex+', 36273), ('now', 34464), ('nm', 32148), ('bowman', 31986), ('on', 31829), ('read', 31508), ('variation', 31137), ('cards', 30950), ('nm/', 30728), ('mini', 29151), ('diamond', 28735), ('collection', 28121), ('break', 27806), ('league', 

In [123]:
import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz','stanford-ner-4.2.0.jar',encoding='utf-8')
def person2(text):
    for sent in nltk.sent_tokenize(text):
        tokens = nltk.tokenize.word_tokenize(sent)
        tags = st.tag(tokens)
        res = ""
        for tag in tags:
            if tag[1]=='PERSON':
                res = res + " "+ tag[0]
    return res.strip()

In [124]:
undetect_name_text= [i[0] for i in most_frequent_words]
undetect_name_text = " ".join(undetect_name_text)
undetected_name = person2(undetect_name_text).split(' ')
undetected_name.extend(["ken","griffey","allen","ginter","ohtani","shohei",
                          "alvarez","franco","lewis","jim",
                          "al","nick","kyle","yordan","gavin","mike","dominguez",
                          "rodriguez","jasson","john","ryan","andrew","alex","diego","mckenzie",
                           "carlson","howard","zac","bryan","clarke","adams","brian","pearson",
                          "alexander","andy","perez","chuck","mackenzie","hernandez","andres",'anderson',
                          "matthew","kopech","rosario","stewart","marte","lee","brandon",'dalquist',
                           "triston","tim",'dylan','larry','martin','derek','pete','abrams','foli',
                     'jamieson','jimenez','amethyst','charley','alston','christin', 'kershaw','luke',
                        'sullivan', 'dustin','shane','bush','ryne', 'mccarthy', 'alec', 'willi','hunter',
                        'samuel','dawson','kelley','jimmie','yogi','tj','rutledge','leo','johnstone',
                        'eli','giles','varsho','ronaldo','mcconnell','sonny','evan','devin','medina',
                        'yoshitomo','kindall','herrnstein','dempsey','clayton','connell','levi',
                        'derrell','jahmai','marvin','blaylock','joba','lincoln','seth','delvin','scioscia',
                        'kenny','kenney','donald','ismael','locke','opitz','janowicz','dereck','burbrink',
                        'blackmon','leron','cade','bieber','lehner','jonathon','kuhn','dahl','jonatan',
                        'jurado','dirk','steiner','kaminsky','dana','dubiel','franklyn','poppel','garko',
                        'adon','jaeckel','quinton','mccracken','ackley','peter','senzel','mason','ferrara',
                       'wander','lux','matt','wallner','spangler','julio','wright','friday','verdugo'])


In [125]:
undetected_name = set(undetected_name)
def person_left(x):
    if not x['Player_Name']:
        res = ""
        for i in x['Title'].split(" "):
            if i in undetected_name:
                res = res +" "+ i
        if res:
            return res.strip()
        else:
            return np.NaN
    else:
        return x['Player_Name']
        

In [126]:
df1['Player_Name'] = df1.apply(lambda x:person_left(x),axis = 1)

In [127]:
df1['Player_Name'].isnull().sum()

528501

In [128]:
def num_words_unmatched(x):
    y = x.split(' ')
    while("" in y): 
        y.remove("")
    if y:
        return len(y)
    else:
        return 0

In [129]:
def cal_matched_percentage(x):
    if x['num_words_unmatched'] == 0:
        return 1
    else:
        return 1-(x['num_words_unmatched']/x['num_words_title'])

In [130]:
df1['Unmatched']=df1.apply(lambda x: else_unmatched(x),axis = 1)

In [ ]:
df1['Unmatched']=df1.apply(lambda x: unmatched_trim(x),axis = 1)

In [142]:
most_frequent_words = FreqDist(lowercase_words).most_common(2000)
junk_words = [i[0] for i in most_frequent_words]
junk_words = set(junk_words)
def match_junks(x):
    res = ""
    for i in x.split(' '):
        if i in junk_words:
            res = res+" "+i
    if res:
        return res.strip()
    else:
        return np.NaN

In [143]:
df1['Junks']=df1['Unmatched'].apply(lambda x: match_junks(x))

In [144]:
df1

,Title,Year,Unmatched,TeamName,Grader,Classifier,CardNumber,QuantityMade,Specialty,Player_Name,Manufacturer,Rating,num_words_unmatched,num_words_title,matched_percentage,Junks
0,1974 laughlin all-star games phil cavarretta #44,1974,all-star games,None,NaN,NaN,44,NaN,,Phil Cavarretta,NaN,NaN,2,7,0.714286,all-star games
1,1955 bowman del crandall #217,1955,,None,NaN,NaN,217,NaN,,Del Crandall,NaN,NaN,0,5,1.000000,NaN
2,1957 topps del crandall #133,1957,,None,NaN,NaN,133,NaN,,Del Crandall,NaN,NaN,0,5,1.000000,NaN
3,1995 skybox emotion timeless cal ripken jr #8 hof,1995,emotion timeless jr,None,NaN,box,8,NaN,hof,Cal Ripken,NaN,NaN,3,9,0.666667,emotion timeless jr
4,1989 st vincent rookies stamps ken griffey jr ...,1989,st vincent stamps jr,None,NaN,NaN,NaN,NaN,"rookies,hof,rookie,rookie",ken griffey,NaN,NaN,4,10,0.600000,st vincent stamps jr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293584,shohei ohtani 2019 bowman sterling black atomi...,2019,sterling black atomic refractor graph,None,NaN,NaN,/10,/10,"auto,auto",shohei ohtani,NaN,NaN,5,12,0.583333,sterling black atomic refractor graph
2293585,shohei ohtani 2019 bowman sterling black atomi...,2019,sterling black atomic refractor graph,None,NaN,NaN,/10,/10,"auto,auto",shohei ohtani,NaN,NaN,5,12,0.583333,sterling black atomic refractor graph
2293586,shohei ohtani 2019 bowman sterling black atomi...,2019,sterling black atomic refractor graph,None,NaN,NaN,/10,/10,"auto,auto",shohei ohtani,NaN,NaN,5,12,0.583333,sterling black atomic refractor graph
2293587,shohei ohtani 2019 bowman sterling black atomi...,2019,sterling black atomic refractor graph,None,NaN,NaN,/10,/10,"auto,auto",shohei ohtani,NaN,NaN,5,12,0.583333,sterling black atomic refractor graph


In [145]:
def replace_junks(x):
    res = x['Unmatched']
    if isinstance(x['Junks'],str):
        for i in x['Junks'].split(' '):
            if i in res:
                res = res.replace(i,"")
    return res

In [146]:
df1['Unmatched']=df1.apply(lambda x: replace_junks(x),axis = 1)

# Calculating Stats

In [147]:
df1['Unmatched']=df1.apply(lambda x: unmatched_trim(x),axis = 1)

In [148]:
df1['num_words_unmatched']=df1['Unmatched'].apply(lambda x: num_words_unmatched(x))

In [149]:
df1['num_words_title']=df1['Title'].apply(lambda x: len(x.split(' ')))

In [150]:
df1['matched_percentage']=df1.apply(lambda x: cal_matched_percentage(x),axis = 1)

In [151]:
df1['matched_percentage'].mean()

0.9546953080999556

In [136]:
def manufacturer_back(x):
    if isinstance(x,str):
        arr = x.split(',')
        res = ""
        while("" in arr) : 
            arr.remove("")
        while(" " in arr) : 
            arr.remove(" ")
        for i in arr:
            res = res + " "+ manufacturer_dict[i.strip()]
        return res
    else:
        return np.NaN

In [137]:
def rating_back(x):
    if isinstance(x,str):
        return rating_dict[x]
    else:
        return np.NaN

In [138]:
def lower_back(x):
    if isinstance(x,str):
        return x.lower()
    else:
        return np.NaN

In [139]:
df1['Manufacturer']=df1['Manufacturer'].apply(lambda x: manufacturer_back(x))
df1['Rating'] = df1['Rating'].apply(lambda x: rating_back(x)) 

In [140]:
# df1['Specialty']=df1['Specialty'].str.upper()
# df1['Classifier']=df1['Classifier'].str.capitalize()
# df1['Grader']=df1['Grader'].str.upper()

In [141]:
df1.to_csv('baseball_df.csv', index=False)